In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
from sklearn.cluster import k_means, kmeans_plusplus, KMeans
from sklearn.decomposition import PCA

# K-Means and Clustering

Clustering algorithms are a part of machine learning that attempts to group, based on the features in the dataset, records into "clusters" of similar examples. Clustering is unsupervised - we don't know which groupings we'll end up with in advance (in contrast to something like logistic regression, where we know going in what the resulting classes are, we are just trying to fit each records into one of those groups)

## Unsupervised Learning

The clustering that we will look at here is an example of unsupervised learning, or machine learning models that do not have predefined labels like we had in classifications. Instead, the model will try to find patterns in the data and group them together based on the similarities in the data itself. We don't start knowing the possible outcomes, we give the data to the algorithm and it tells us what the groupings should be. 

### Clustering

Clustering is simple in concept, the goal is to group our data points into clusters, or groups of records that are the most similar to each other, and the most dissimilar to the records in other clusters. The main difference from classification is that we don't know what the clusters are going to be in advance. We feed in data, normally set the number of separate groups we want, and the algorithm figures out what the groups should be.

![Clustering](images/clustering.png "Clustering")

Clustering algorithms are common in the real world, in several applications. They are very helpful when we don't really know the end results that we are expecting. Some common applications of clustering algorithms are:
<ul>
<li> Customer segmentation - grouping customers into different groups based on their purchasing patterns, demographics, etc.
<li> Medical diagnosis - grouping patients into different groups based on their symptoms, etc.
<li> Social network analysis - grouping people into different groups based on their social connections, etc.
<li> Anomaly detection - looking for things that <i>don't</i> fit into the other groups well, as they may be things to investigate. 
</ul>

Clustering logic is also commonly used in conjuction with other algorithms. For example, we can use clustering to group our data into different groups, and then use classification on each group to get a more accurate result. We can also use it to help with smarter imputation. 

### My First Cluster! Building to K-Means Clustering

We can cluster data using K-Means, first, we can make some simple data to cluster. We will use the make_blobs function to make 300 samples with 2 features, X and Y, which fall into 4 distinct clusters. We will then plot the data to see what it looks like.

In [ ]:
from sklearn.datasets._samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50)

##### Visually Clustering our Blobs

We can look at the dummy data plotted above and it is pretty clearly broken up into 4 blobs that we can identify visually pretty easily. 

![Visual Clustering](images/visual_clusters.png "Visual Clustering")

Each group of points that I have circled is one "cluster" - here we eyeballed it; clustering algorithms will try to do this automatically by finding the points that are <b><i>most similar</i></b> to each other based on their features, which are X and Y here. 

#### Clustering with K-Means

We can use kmeans to create clusters similar to the above, but with some slightly more formal clustering than my freehand lines. Clustering works by finding these blobs of similar points, and grouping them into "like" groups similarly to how we do by eye. 

##### K-Means Clusters

The mechanics of this are very simple, we'll use the KMeans clustering function very similarly to how we use any model in scikit-learn. We'll create a KMeans object, and then fit it to our data, then generate predictions for each point in the data - or assign it to a cluster. The "Cluster" column that we see below is which of the 4 clusters that we saw above that each point is assigned to. The unsupervised factor here is that we don't have any idea which cluster is which, the alogrithm will assign them to the clusters that it thinks are most similar, but which group is "1', which is "2", etc. is unknown to us going in.

In clustering we don't really have the train/test split, because we don't have anything to test against. We just have the data, and we want to find the clusters - there's not right or wrong answer. 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
clusters = kmeans.predict(X)

## Create DataFrame with cluster labels for display
testdf = pd.DataFrame(X)
testdf['Cluster'] = clusters
testdf.head()

##### Labels

Since our model was fitted to our data, we can shortcut the prediction step and just get the label with the labels_ attribute. This will give us the cluster that each point was assigned to, just as the prediction step did above. When we are using all of our data for fitting, this is exactly the same. 

<b>Note:</b> One important thing to remember in clustering is that the labels that the clustering algorithm assigns is just a number, it has no meaning outside of differentiating the clusters. So being in cluster 1 or 0 doesn't equate to True/False or any other external label. Which clustered group gets which label is 100% arbitrary. 

In [ ]:
clusters2 = kmeans.labels_

testdf2 = pd.DataFrame(X)
testdf2['Cluster'] = clusters2
testdf2.head()

##### Plot Clusters

We can now plot the clusters that we have created, along with the center point of each cluster. 

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=clusters, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)


## Creating K-Means Clusters and Inertia

Much like logistic regression and gradient descent, clustering relies on an iteritive algorithm to find the corect clustering. In this case the algorithm attempts to minimise a metric called inertia. Inertia is the within-cluster sum-of-squares, roughly the compactness of each cluster. 

$ \sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)  $

This calculation is just the sum of squares distance of each point to the center of that group. The algorithm is pretty simple as well:
<ul>
<li> Start with a random set of centroids.
<li> Assign each point to the closest centroid. 
<li> Calculate the inertia (above).
<li> Move the centroids to the center of the points that are assigned to that centroid. 
<li> Recalculate the inertia.
<li> Repeat this process until the inertia stops changing.
</ul>

![K-Means](images/kmeansMath.png "K-Means")

![K-Means Process](images/kmeansViz.png "K-Means Process")

While using this metric of inertia generates "tight" groupings, there are a few concerns:
<ul>
<li> The distance from the center point is radial. If there is an irregular pattern to the data distribution, this may not make sense. 
<li> When using high dimension data, the distances can inflate. Using an algorithm to reduce dimensionality like PCA can help this. 
</ul>

The cluster centers are randomly placed, then the inertia is calculated and the points are updated. This process uses something called Expectation Maximization, but the effect is exactly as though we were doing gradient descent with a cost function - eventually we converge on the solution. One issue is that, like with some gradient descent problems, we can have the optimized solution fall into a local minima, not the global solution. To combat this the kMeans built into sklearn defaults to using 10 iterations, in each one the starting point of the centroids is different, so we can likely avoid this. We can also call a paramater called "init" which changes the starting placement of the centroids from random to "k-means++" which looks at the data to generate smart starting points to speed convergance on a solution, and avoid local minima. 

We don't need to delve super deep into the math of how things work here, as long as we understand that the algorithm iteritively persues the best solution, or the lowest interita, we are good. 

### Clustering Example

Using real data, we'll cluster the diabetes havers/not havers based on their metrics. 

In [ ]:
#Load Data
df = pd.read_csv("data/diabetes.csv")
df.drop(columns={"Outcome"}, inplace=True)
df.head()

### Scale and PCA Data

Here we'll scale the data and also run it through PCA so we can project it into 2D for visualization. Our data is currently in 8 dimensions - we can't really draw that effectively. Using PCA to make visualization easier for high dimension data is pretty common. For example, song data is analyzed by people like Spotify, and they produce visualizations to illustrate aspects of their analysis:

![PCA Visualization](images/pca_cluster_songs.png "PCA Visualization")

These projections down to 2 dimensions for visualizing data are hit and miss in terms of how "good" they are at showing the data. There is obvoiusly a lot of information lost in the process, but the alternative is not being able to visualize the data at all. When looking at clustering, these visualizations are pretty common, as seeing the differnet clusters is something you probably want to be able to do.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_clus = MinMaxScaler().fit_transform(df)
X_c = PCA(2).fit_transform(X_clus)

### Plot Clusters

We need to specify the number of clusters to produce. We can plot several selections of k and see visually how well that fits our data. 

In [ ]:
nc = [2,3,4,5,6,7]
fig, ax = plt.subplots(2, 3, figsize=(15, 9))
row = 0
col = 0
for n in nc:
    kmeans = KMeans(n_clusters=n, max_iter=1000)
    kmeans.fit(X_c)
    y_kmeans = kmeans.predict(X_c)
    ax[row][col].scatter(X_c[:,0], X_c[:,1], c=y_kmeans, s=50, cmap='viridis', alpha=.2)
    centers = kmeans.cluster_centers_
    #print(centers)
    ax[row][col].scatter(centers[:, 0], centers[:, 1], c='black', s=20)
    col += 1
    if col >= 3:
        col = 0
        row += 1
plt.show()


## How Many Clusters Should I Have?

How many clusters should we have? If our data is easy to visualize, we can just look at it. If it isn't, we need some analysis. Two ways are:
<ul>
<li> Elbow method - like with PCA, we can plot the inertia and choose where it flattens. Just like the scree plot. 
<li> Silhouette - measure of how similar points are within a cluster compared to the distance from other clusters. Basically a measure of separation between clusters, analagous to the division boundaries we looked at with SVMs. We want high silhouette scores. 
    <ul>
    <li><b> Note: this is a little more elaborate, we'll look at the scores, but stick with the elbow method for now. </b>
    </ul>
</ul>

These methods can be used in cocert with each other to determine optimal number of clusters. 

In [ ]:
# Elbow Method
Sum_of_squared_distances = []
K = range(1,10)
for num_clusters in K :
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X_c)
    Sum_of_squared_distances.append(kmeans.inertia_)
plt.plot(K,Sum_of_squared_distances,"bx-")
plt.xlabel("Values of K") 
plt.ylabel("Sum of squared distances/Inertia") 
plt.title("Elbow Method For Optimal k")
plt.show()

In [ ]:
#Silhouette Score
from sklearn.metrics import silhouette_score
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
silhouette_avg = []
for num_clusters in range_n_clusters:
    # initialise kmeans
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X_c)
    cluster_labels = kmeans.labels_
    # silhouette score
    silhouette_avg.append(silhouette_score(X_c, cluster_labels))
plt.plot(range_n_clusters,silhouette_avg,"bx-")
plt.xlabel("Values of K") 
plt.ylabel("Silhouette score") 
plt.title("Silhouette analysis For Optimal k")
plt.show()

### Example - Clustering Digits

We can try to cluster the digits that we looked at previously. We can try to group the drawings into digits without using their labels. 

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
Xp = digits.data
yp = digits.target

For this we know that we'll need 10 clusters (think about this number a bit...), so that part is easy. 

In [ ]:
kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(Xp)
kmeans.cluster_centers_.shape


#### Plot the "Average" Digit

We can plot the center of each cluster. These centers are each 64 dimension points - one for each feature. The center is the middle of each cluster, so it isn't the "most average" one of the images that we started with, it is the average of all the pixels of each drawing of that number. Or more accurately the average of all the pixels of each digit that got clustered together - correctly or incorrectly. We are creating a new image, the one that is the actual average of all the images in that cluster.

This concept is something that we'll use more later - once we fit a clustering algorithm to our data, the model "knows" what each cluster "looks like".

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = kmeans.cluster_centers_.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

### Digits are Grouped!

We don't know the classes going in, but our clustering has managed to group the digits pretty well! 

#### But What ARE the Predictions?

Clustering doesn't know anything about the labels of our digits, all it is doing is grouping together the similar pieces of data and giving each group some arbitrary label - group 1, group 2, etc... To figure out which are correct or incorrect we need to label each group. To do so, we'll take the mode, or most frequent, from the set of labels. We can then check an entire group against the correct label. This is a key characteristic of unsupervised learning - we aren't putting things into classes that we know, like a classification model, we are looking <i>only</i> at the data, and putting that data into groups, each of which we don't know what that group is, only that the elements in it are similar. 

We generally don't know labels at when clustering, but since this is a trial run, we do. We can pull those in to check our accuracy. This isn't a standard action, we are just doing a check. 

In [ ]:
from scipy.stats import mode

labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(yp[mask])[0]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(yp, labels)

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(digits.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')

### Results

We didn't do too badly! Most errors are what we'd expect, especially with such low resolution digits. Things that are similar, like skinny 8s and 1s, or 8s and 9s are often confused, other than those we are quite accurate. 

But can we do better...

## t-Distributed Stochastic Neighbor Embedding 

t-SNE is t-Distributed Stochastic Neighbor Embedding - a dimension reduction technique that is non-linear, probabilistic, and typically used to project high dimension data to a 2D plot. tSNE is new - invented in 2008. 

tSNE does the same basic thing as PCA, it takes a dataset that has a high number of dimensions and translates it to a lower number of dimensions. The math behind the scenes is different, and pretty complex. tSNE is a type of manifold learning - or non-linear dimension reduction - in contrast to PCA and LSA which are both linear techniques. This gives tSNE an advantage that it is often able to better project high dimension data in a way that makes good visualizations - which is the primary use. We'll look at tSNE a little more when we do larger data with neural networks. Here, we can use tSNE as a dimension reduction tool just as we would PCA. 

#### How tSNE Works

tSNE very roughly measures the distance between points in space - N dimensional space where N is the dimensionality of the data. These distances are then used with a t-distribution (hence the t) to generate probabilities of the relative similarities between points. These are then mapped down to 2 (usually) dimensions randomly, based on the probability. This is a very loose generalization, details are here: https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a but it goes well beyond what we need to care about. We can treat tSNE as a more flexible PCA here and cut our dimensions from 64 to 2. 

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
digits_proj = tsne.fit_transform(digits.data)

# Compute the clusters
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits_proj)

# Permute the labels
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

# Compute the accuracy
accuracy_score(digits.target, labels)

#### Plot tSNE Projections

We can plot the tSNE projections for the clusters. Since the data is in 2D, we can visualize it.  

In [ ]:
plt.scatter(digits_proj[:,0], digits_proj[:,1], c=clusters, s=50, cmap='viridis', alpha=.2)
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=20)

### tSNE and Plotting High Dimension Data and Perplexity

As an example, we can plot something in lots of dimensions into fewer - the breast cancer dataset. This dataset has 30 dimensions, so doing a scatter plot of the data the "normal" way would require just selecting 2 dimensions to use. Here we can project those 30 dimensions down to 2, so we can visualize our data. 

In [ ]:
from sklearn.datasets import load_breast_cancer
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

df = sklearn_to_df(load_breast_cancer())
y1 = df["target"]
X1 = df.drop(columns="target")
red = y1 == 0
green = y1 == 1
X1 = MinMaxScaler().fit_transform(X1)
df.head()

### Projection with PCA

First, we can use PCA to reduce this to 2 dimensions and see what we get. We'll color the data according to it's true label, so we can visually see the separation when plotted. 

In [ ]:
#PCA
b_PCA = PCA(2).fit_transform(X1)
plt.scatter(b_PCA[red,0], b_PCA[red,1],s=50, c='r', alpha=.7)
plt.scatter(b_PCA[green,0], b_PCA[green,1], s=50, c='g', alpha=.7)

### Now tSNE

The tSNE projections will vary, you may get one that is quite different. 

In [ ]:
# tSNE
#b_tSNE = TSNE(n_components=2, init='random', perplexity = 30).fit_transform(X1)
b_tSNE = TSNE(n_components=2, perplexity = 5).fit_transform(X1)
plt.scatter(b_tSNE[red,0], b_tSNE[red,1],s=50, c='r', alpha=.7)
plt.scatter(b_tSNE[green,0], b_tSNE[green,1], s=50, c='g', alpha=.7)

### Perplexity

Perplexity is an argument for tSNE, it can radically change the projection that we get. Perplexity roughly controls how local vs global the algorithm looks when trying to find structure in the data. Higher perplexities cause the algorithm to look more globally at the data - so it will look for relationships between points that are "farther apart" (in N-dimensional space). Lower perplexities cause the algorithm to look more locally. Typical values are around 5 to 50, when visualizing, we kind of need to guess and check. There is a very good explainer of tSNE along with many good visualizations here: https://distill.pub/2016/misread-tsne/ 

As the perplexity changes we can sometimes get shapes and patterns in the data. 

In [ ]:
#Plot several perplexities
fig, ax = plt.subplots(2, 3, figsize=(15, 9))
row = 0
col = 0
perp = [1,25,50,75,100,150]
for p in perp:
    b_tSNE = TSNE(n_components=2, init='random', perplexity = p).fit_transform(X1)
    ax[row][col].scatter(b_tSNE[red,0], b_tSNE[red,1],s=50, c='r', alpha=.7)
    ax[row][col].scatter(b_tSNE[green,0], b_tSNE[green,1], s=50, c='g', alpha=.7)
    col += 1
    if col >= 3:
        col = 0
        row += 1
plt.show()

In [ ]:
#### Cluster the Breast Cancer Data
b_tSNE = TSNE(n_components=2, init='random', perplexity = 30).fit_transform(X1)
kmeans = KMeans(n_clusters=2, random_state=0)
clusters = kmeans.fit_predict(b_tSNE)
plt.scatter(b_tSNE[:,0], b_tSNE[:,1],s=50, c=clusters, alpha=.7)

### What Should the Perplexity Be?

This is a question that we only care a little bit about, and don't have a conclusive rule for. The 5 to 50 guideline above is a good starting point, for large datasets we may want to try higher values. We aren't going to get too far into tuning the tSNE results, if we are using it for dimensional reduction, we can try a few values in a grid search and see what works best. There is a comprehensive article on tunin tSNE here, but it really isn't something that we'll be focusing on: https://towardsdatascience.com/how-to-tune-hyperparameters-of-tsne-7c0596a18868 

#### Number of Components

tSNE is very commonly used to reduce data to 2 dimensions so we can visualize high dimension data. If we are using tSNE to do dimension reduction, we can use the same methods as we did with PCA to determine the number of clusters. 

In [ ]:
#Silhouette Scores
from sklearn.metrics import silhouette_score
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
silhouette_avg = []
for num_clusters in range_n_clusters:
    # initialise kmeans
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X1)
    cluster_labels = kmeans.labels_
    # silhouette score
    silhouette_avg.append(silhouette_score(X1, cluster_labels))
plt.plot(range_n_clusters,silhouette_avg,"bx-")
plt.xlabel("Values of K") 
plt.ylabel("Silhouette score") 
plt.title("Silhouette analysis For Optimal k")
plt.show()

### tSNE Conclusion

tSNE is basically a functional equivalent to PCA that does not depend on linear relationships. 

We can use it to reduce dimensions just like PCA. Generally, we'd want to try it after PCA, or if PCA seems to not do a great job with our data. It is way slower

The primary use of tSNE is to visualize high dimension data in 2D, the structure in the data that it is able to capture tends to make it better suited to creating useful projections. tSNE has a few considerations:
<ul>
<li> tSNE can be very slow, while PCA is comparatively fast. This can be a big limitation on large datasets if PCA is used to reduce dimensions for modelling. 
<li> When using tSNE with massive numbers of features, it may be practical to use PCA first, to reduce the dimensions down to 50ish. 
<li> Sriously, it is slow with lots of data. 
</ul>

In the manifold learning package of sklearn there are other dimension reduction algorithms that do the same thing. Each of these may have different strengths and weaknesses when dealing with different types of data. If we are reducing dimensions we probably want to start with PCA for dimension reduction, tSNE for visualization, and move on to something else if needed. 

## Exercise - Clustering Wine

Cluster the wine. Customer_Segment is the target, so we'll drop that for clustering. 
<ul>
<li> Project a plot with tSNE, I used a perplexity of 8 and got good results - try a few. 
<li> Use KMeans to cluster the data.
</ul>

In [ ]:
wine = pd.read_csv("data/Wine.csv")
wine.head()

In [ ]:
#Plot


### Plot with Different Number of Clusters

We can cluster our data with a few different numbers of clusters to see what we get. 

#### Check the "Correct" Number of Clusters

I will use a silhouette to see what the best number of clusters is. Then check the actual labels from the data to compare. 

In [ ]:
#Silhouette Scores
